# 1. Импортирование библиотек и модулей

In [235]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
import joblib

# 2.Загрузка обучающей и тестовой выборок

In [236]:
fname_tr = "datasets/HDAC6_ws.sdf"

mols_tr = []
y_tr = []
for mol in Chem.SDMolSupplier(fname_tr):
    if mol is not None:
        mols_tr.append(mol)
        y_tr.append(mol.GetIntProp("Active"))

In [237]:
fname_ts = "datasets/HDAC6_ts.sdf"

mols_ts = []
y_ts = []
for mol in Chem.SDMolSupplier(fname_ts):
    if mol is not None:
        mols_ts.append(mol)
        y_ts.append(mol.GetIntProp("Active"))

# 3.Расчет дескрипторов обучающей выборки

In [238]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
header = calc.GetDescriptorNames()

In [239]:
descr_tr= []
for m in mols_tr:
    descr_tr.append(calc.CalcDescriptors(m))
x_tr = np.asarray(descr_tr)

In [240]:
df_RDKit_2D = pd.DataFrame(x_tr,columns=header)

In [241]:
df_RDKit_2D.head(2)

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,12.330332,-0.593978,12.330332,0.044291,0.366637,302.374,280.198,302.163043,118.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12.715980,-3.663292,12.715980,0.213972,0.432827,342.376,328.264,342.067428,122.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [242]:
x_tr.shape

(105, 208)

# 4. Расчет дескрипторов тестовой выборки

In [243]:
descr_ts = []
for m in mols_ts:
    descr_ts.append(calc.CalcDescriptors(m))
x_ts = np.asarray(descr_ts)

In [244]:
x_ts.shape

(26, 208)

# 5. Построение и валидация модели RF 

## 5.1. Построение модели RF 

In [245]:
seed = 42

In [246]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [247]:
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3],
              "n_estimators": [100, 250, 500, 1000]}

In [248]:
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [249]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   11.5s
[Parallel(n_jobs=2)]: Done  80 out of  80 | elapsed:   23.0s finished
C:\Users\08A4~1\AppData\Local\conda\conda\envs\my-rdkit-env\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
           

In [250]:
m.best_params_

{'max_features': 20, 'n_estimators': 500}

In [251]:
best_clf_RF = m.best_estimator_

## 5.2. 5-fold-cross-validation  модели RF

In [252]:
y_pred_CV_RF = cross_val_predict(best_clf_RF, x_tr, y_tr, cv=cv)

In [253]:
confusion_matrix_RF = metrics.confusion_matrix(y_tr, y_pred_CV_RF, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_RF, weights='linear')
TN, FP, FN, TP = confusion_matrix_RF.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.8
SE =  0.68
SP =  0.93
Kappa =  0.63


In [254]:
joblib.dump(best_clf_RF, "models/HDAC6_RF_RDKit_2D.pkl", compress=3)

['HDAC6_long_list_MGU/20  march 2022 without scale/HDAC6_RF_RDKit_2D.pkl']

## 5.3.Y-randomization для модели RF

In [255]:
permutations = 20
score, permutation_scores, pvalue = permutation_test_score(best_clf_RF, x_tr, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


True score =  0.81 
Y-randomization =  0.48 
p-value =  0.0476


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   15.2s finished


## 5.4. Валидация на внешней тестовой выборке для модели RF

In [256]:
y_pred_rf = best_clf_RF.predict(x_ts)

In [257]:
y_pred_rf

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0])

In [258]:
confusion_matrix_ts = metrics.confusion_matrix(y_ts, y_pred_rf, labels=[0,1])

In [259]:
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_rf, weights='linear')
TN, FP, FN, TP = confusion_matrix_ts.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.92
SE =  0.89
SP =  0.94
Kappa =  0.83


## 5.5. Выявление 5 наиболее значимых признаков

In [260]:
imp = best_clf_RF.feature_importances_

In [261]:
indices = np.argsort(imp)[::-1]

In [262]:
print("Feature ranking:")

# print top 5 features
for i in range(5):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

Feature ranking:
1. feature 48 (0.046026)
2. feature 22 (0.036923)
3. feature 100 (0.031736)
4. feature 17 (0.029979)
5. feature 3 (0.027863)


In [263]:
df_RDKit_2D.iloc[:, [48, 22, 100, 17, 3]]

,PEOE_VSA12,BCUT2D_LOGPLOW,VSA_EState7,BCUT2D_MWHI,MinAbsEStateIndex
0,0.000000,-2.148483,4.616686,16.466793,0.044291
1,0.000000,-2.142886,4.202210,32.233325,0.213972
2,11.814359,-2.172474,5.082345,19.145439,0.130424
3,11.814359,-2.172474,5.412180,16.466980,0.091429
4,11.814359,-2.172474,5.626235,16.466936,0.059463
...,...,...,...,...,...
100,5.907180,-2.181693,2.412524,16.475853,0.027149
101,5.907180,-2.273960,2.902825,16.644506,0.018391
102,5.907180,-2.181694,2.342483,16.475866,0.022401
103,5.907180,-2.325164,-0.581986,16.566778,0.019060


# 6. Расчет AD методом Euclidian distances при K=1 (https://doi.org/10.1021/acs.jcim.0c00415)

In [264]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [265]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,95,96,97,98,99,100,101,102,103,104
0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2.924382e+04,3.570522e+02,5.281930e+05,9.300985e+06,4.889977e+01,2.262601e+04,1.535351e+06,1.743071e+06,3.160302e+04,1.503897e+04,...,5.832893e+05,1.711208e+01,6.766899e+04,3.235252e+05,1.283203e+06,8.562572e+01,8.562572e+01,2.729952e+05,9.105191e+06,5.468408e+03
2,3.471071e+04,3.571113e+02,9.028959e+05,1.040775e+07,1.842330e+06,3.160302e+04,3.969924e+07,2.849838e+06,3.795672e+04,3.198289e+04,...,1.902754e+06,3.571113e+02,6.766902e+04,4.524404e+05,1.319465e+06,8.109176e+03,8.109629e+03,3.099402e+05,6.967719e+07,2.924382e+04
3,3.719271e+04,1.947552e+04,1.671934e+06,1.215082e+07,1.983286e+06,6.955962e+04,9.529116e+07,6.705018e+06,5.422840e+04,3.198290e+04,...,2.626336e+06,1.983036e+04,8.103568e+04,5.342476e+05,1.902754e+06,3.905668e+04,3.905663e+04,4.220554e+05,2.830270e+08,4.452387e+04
4,7.376723e+04,3.233739e+04,2.031626e+06,1.474295e+07,2.364037e+06,1.083748e+05,1.059526e+08,8.068986e+06,8.378398e+04,3.233739e+04,...,3.185958e+06,3.198290e+04,1.824740e+06,5.342476e+05,3.755540e+06,4.452407e+04,4.452387e+04,5.608618e+05,4.885061e+08,4.452407e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1.872691e+09,1.872370e+09,1.856238e+09,1.822425e+09,1.860683e+09,1.872235e+09,1.699929e+09,1.834576e+09,1.872266e+09,1.872402e+09,...,1.848303e+09,1.872370e+09,1.866887e+09,1.864739e+09,1.846401e+09,1.872617e+09,1.872617e+09,1.869520e+09,1.220242e+09,1.872662e+09
101,2.419646e+09,2.419324e+09,2.403192e+09,2.369380e+09,2.407637e+09,2.419189e+09,2.246884e+09,2.381531e+09,2.419221e+09,2.419356e+09,...,2.395258e+09,2.419324e+09,2.413842e+09,2.411694e+09,2.393355e+09,2.419572e+09,2.419572e+09,2.416474e+09,1.220279e+09,2.419616e+09
102,2.446919e+09,2.446597e+09,2.430465e+09,2.396653e+09,2.434910e+09,2.446462e+09,2.274157e+09,2.408803e+09,2.446494e+09,2.446629e+09,...,2.422531e+09,2.446597e+09,2.441115e+09,2.438967e+09,2.420628e+09,2.446845e+09,2.446845e+09,2.443747e+09,1.226677e+09,2.446889e+09
103,2.939493e+09,2.939172e+09,2.923040e+09,2.889227e+09,2.927485e+09,2.939037e+09,2.766732e+09,2.901378e+09,2.939068e+09,2.939204e+09,...,2.915106e+09,2.939172e+09,2.933690e+09,2.931541e+09,2.913203e+09,2.939420e+09,2.939420e+09,2.936322e+09,1.719251e+09,2.939464e+09


In [266]:
similarity= neighbors_k

In [267]:
Dmean=np.mean(similarity[1,:])

In [268]:
round(Dmean, 2)

24908205.68

In [269]:
std=np.std(similarity[1,:])

In [270]:
round(std, 2)

112597654.25

In [271]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

81207032.8


In [272]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [273]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,2.274629e+02,2.493019e+06,2.512593e+01,3.543677e+04,1.174441e+04,6.973401e+03,2.022698e+05,4.080051e+07,1.889487e+07,4.885820e+01,...,6.652448e+04,8.716560e+03,3.358001e+03,1.304004e+05,1.184349e+05,3.513458e+04,1.489620e+04,1.007968e+02,4.872662e+03,8.688165e+01
1,2.305302e+02,1.086620e+08,1.626308e+02,6.071169e+04,2.621267e+04,1.565330e+04,1.195211e+06,7.840534e+07,2.043022e+07,8.789788e+03,...,7.443066e+04,1.763302e+04,1.927666e+04,1.363779e+05,3.036207e+05,1.161698e+05,5.278102e+04,3.235252e+05,8.668025e+04,1.520190e+02
2,1.842330e+06,1.101973e+08,1.628419e+02,6.848104e+04,4.334708e+04,3.857638e+04,2.233896e+06,2.230608e+08,5.859411e+07,8.916855e+03,...,5.735884e+05,2.642228e+04,5.087733e+04,1.938235e+05,5.766159e+05,1.838387e+05,5.278103e+04,4.524404e+05,8.668025e+04,8.110655e+03
3,1.983286e+06,1.483612e+08,3.274706e+04,7.256862e+04,6.597246e+04,7.653297e+04,2.762089e+06,2.297442e+08,8.727408e+07,9.453946e+04,...,7.780664e+05,1.121716e+05,8.883346e+04,2.203386e+05,6.051943e+05,1.838387e+05,9.592662e+04,5.342476e+05,8.668086e+04,3.905675e+04
4,2.364037e+06,1.864507e+08,3.274708e+04,9.194950e+04,7.204002e+04,1.014014e+05,3.073909e+06,3.359131e+08,1.141860e+08,1.171645e+05,...,1.436493e+06,1.347968e+05,8.910204e+04,3.803380e+05,6.135608e+05,1.940909e+06,1.809850e+06,5.342476e+05,1.194235e+05,4.452379e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,1.860683e+09,1.591268e+09,1.864205e+09,1.871658e+09,1.872278e+09,1.872228e+09,1.854004e+09,1.364016e+09,1.681035e+09,1.872118e+09,...,1.862592e+09,1.872100e+09,1.872216e+09,1.870649e+09,1.869824e+09,1.867004e+09,1.866873e+09,1.864739e+09,1.864292e+09,1.872617e+09
101,2.407637e+09,2.138222e+09,2.411159e+09,2.418612e+09,2.419232e+09,2.419182e+09,2.400958e+09,1.910971e+09,2.227989e+09,2.419072e+09,...,2.409546e+09,2.419054e+09,2.419170e+09,2.417603e+09,2.416778e+09,2.413958e+09,2.413827e+09,2.411694e+09,2.411246e+09,2.419572e+09
102,2.434910e+09,2.165495e+09,2.438432e+09,2.445885e+09,2.446505e+09,2.446455e+09,2.428231e+09,1.938244e+09,2.255262e+09,2.446345e+09,...,2.436819e+09,2.446327e+09,2.446443e+09,2.444876e+09,2.444051e+09,2.441231e+09,2.441100e+09,2.438967e+09,2.438519e+09,2.446845e+09
103,2.927485e+09,2.658070e+09,2.931007e+09,2.938460e+09,2.939080e+09,2.939030e+09,2.920806e+09,2.430818e+09,2.747837e+09,2.938920e+09,...,2.929394e+09,2.938902e+09,2.939018e+09,2.937451e+09,2.936626e+09,2.933806e+09,2.933675e+09,2.931541e+09,2.931094e+09,2.939420e+09


In [274]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.27463000e+02 2.49301876e+06 2.51260000e+01 3.54367660e+04
 1.17444140e+04 6.97340100e+03 2.02269847e+05 4.08005065e+07
 1.88948735e+07 4.88580000e+01 5.20621500e+03 1.30048170e+04
 9.65579840e+04 5.24395114e+06 3.80880000e+01 3.57806048e+05
 6.65244850e+04 8.71656000e+03 3.35800100e+03 1.30400447e+05
 1.18434854e+05 3.51345830e+04 1.48962000e+04 1.00797000e+02
 4.87266200e+03 8.68820000e+01]


In [275]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [276]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  1.0


In [277]:
print("Индексы соединений, вошедших в AD = ", np.where(cpd_AD != 0)[0])

Индексы соединений, вошедших в AD =  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]


# 7. Построение и валидация модели GBM

## 7.1. Построение модели GBM

In [278]:
param_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   param_grid, n_jobs=2, cv=cv, verbose=1)

In [279]:
gbm.fit(x_tr, y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  25 out of  25 | elapsed:    2.6s finished
C:\Users\08A4~1\AppData\Local\conda\conda\envs\my-rdkit-env\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score='raise-deprecating',
             estimator=GradientBoostingClassifier(criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=0.5,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_ch

In [280]:
gbm.best_params_

{'n_estimators': 200}

In [281]:
best_clf_GBM = gbm.best_estimator_

## 7.2.  5-fold-cross-validation  модели GBM

In [282]:
y_pred_CV_GBM = cross_val_predict(best_clf_GBM, x_tr, y_tr, cv=cv)

In [283]:
confusion_matrix_CV_GBM = metrics.confusion_matrix(y_tr, y_pred_CV_GBM, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_GBM, weights='linear')
TN, FP, FN, TP = confusion_matrix_CV_GBM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.8
SE =  0.74
SP =  0.87
Kappa =  0.61


In [284]:
joblib.dump(best_clf_GBM, "models/HDAC6_GBM_RDKit_2D.pkl", compress=3)

['HDAC6_long_list_MGU/20  march 2022 without scale/HDAC6_GBM_RDKit_2D.pkl']

## 7.3.Y-randomization для модели GBM

In [285]:
permutations = 20
score, permutation_scores, pvalue = permutation_test_score(best_clf_GBM, x_tr, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


True score =  0.78 
Y-randomization =  0.48 
p-value =  0.0476


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.9s finished


## 7.4. Валидация на тестовой выборке для модели GBM

In [286]:
y_pred_gbm = best_clf_GBM.predict(x_ts)

In [287]:
y_pred_gbm

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0])

In [288]:
confusion_matrix_GBM = metrics.confusion_matrix(y_ts, y_pred_gbm, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_gbm, weights='linear')
TN, FP, FN, TP = confusion_matrix_GBM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.89
SE =  0.89
SP =  0.88
Kappa =  0.75


# 8. Построение и валидация модели SVM

## 8.1. Построение модели SVM

In [289]:
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [290]:
joblib.dump(scale, "models/HDAC_ws_for SVM.pkl", compress=3)

['HDAC6_long_list_MGU/20  march 2022 without scale/HDAC_ws_for SVM.pkl']

In [291]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [292]:
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [293]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 150 out of 150 | elapsed:    1.2s finished
C:\Users\08A4~1\AppData\Local\conda\conda\envs\my-rdkit-env\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=True, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [294]:
best_clf_SVM = svm.best_estimator_

## 8.2. 5-fold-cross-validation модели SVM

In [295]:
y_pred_CV_SVM = cross_val_predict(best_clf_SVM, x_tr, y_tr, cv=cv)

In [296]:
confusion_matrix_CV_SVM = metrics.confusion_matrix(y_tr, y_pred_CV_SVM, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_CV_SVM, weights='linear')
TN, FP, FN, TP = confusion_matrix_CV_SVM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.81
SE =  0.74
SP =  0.88
Kappa =  0.62


## 8.3.Y-randomization для модели SVM

In [297]:
permutations = 20
score, permutation_scores, pvalue = permutation_test_score(best_clf_SVM, x_tr, y_tr,
                                                           cv=cv, scoring='balanced_accuracy',
                                                           n_permutations=permutations,
                                                           n_jobs=-1,
                                                           verbose=1,
                                                           random_state=24)
print('True score = ', score.round(2),
      '\nY-randomization = ', np.mean(permutation_scores).round(2),
      '\np-value = ', pvalue.round(4))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


True score =  0.81 
Y-randomization =  0.49 
p-value =  0.0476


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.9s finished


## 8.4. Валидация на тестовой выборке для модели SVM

In [298]:
scale = joblib.load("models/HDAC_ws_for SVM.pkl")
x_ts = scale.transform(x_ts)

In [299]:
y_pred_SVM = best_clf_SVM.predict(x_ts)

In [300]:
y_pred_SVM

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0])

In [301]:
confusion_matrix_SVM = metrics.confusion_matrix(y_ts, y_pred_SVM, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, y_pred_SVM, weights='linear')
TN, FP, FN, TP = confusion_matrix_SVM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.92
SE =  0.89
SP =  0.94
Kappa =  0.83


# 9. Консенсусное моделирование

## 9.1.  5-fold CV

In [302]:
y_pred_cv_con = 1 * (((y_pred_CV_RF + y_pred_CV_GBM + y_pred_CV_SVM) / 3) >= 0.5)

In [303]:
confusion_matrix_cv_con = metrics.confusion_matrix(y_tr, y_pred_cv_con, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_tr, y_pred_cv_con, weights='linear')
TN, FP, FN, TP = confusion_matrix_cv_con.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.81
SE =  0.71
SP =  0.91
Kappa =  0.64


## 9.2. Test set

In [304]:
pred_c = 1 * (((y_pred_rf + y_pred_gbm + y_pred_SVM) / 3) >= 0.5)

In [305]:
pred_c

array([1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0])

In [306]:
confusion_matrix_GBM = metrics.confusion_matrix(y_ts, pred_c, labels=[0,1])
Kappa = metrics.cohen_kappa_score(y_ts, pred_c, weights='linear')
TN, FP, FN, TP = confusion_matrix_GBM.ravel()
SE = TP/(TP+FN)
SP = TN/(TN+FP)
BA = (SE + SP)/2
print("balanced_accuracy = ", round((BA), 2))
print("SE = ", round((SE), 2))
print("SP = ", round((SP), 2))
print("Kappa = ", round((Kappa), 2))

balanced_accuracy =  0.92
SE =  0.89
SP =  0.94
Kappa =  0.83
